In [1]:
import pandas as pd
import numpy as np

## For plotting
import matplotlib.pyplot as plt
import seaborn as sns

## This sets the plot style
## to have a grid on a dark background
sns.set_style("whitegrid")

In [2]:

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA

from sklearn.metrics import accuracy_score, confusion_matrix




In [3]:
# Read in your csv file that has the mid features. 

#data = pd.read_csv('../Feature Extraction/midFeaturesTrainSet.csv')
data = pd.read_csv('midFeaturesTrainFinal.csv')


data.head()

print(data.shape)

(4876, 140)


In [4]:
# Split the data into train and test set, stratified by Emotion.


data_train, data_test = train_test_split(data.copy(),
                                   shuffle=True,
                                   random_state=608,
                                   stratify=data.Emotion,
                                   test_size=0.2
                                   )

In [5]:
data.sample(5)

,FileID,actorID,Emotion,SentenceID,zcr_mean,energy_mean,energy_entropy_mean,spectral_centroid_mean,spectral_spread_mean,spectral_entropy_mean,...,delta chroma_4_std,delta chroma_5_std,delta chroma_6_std,delta chroma_7_std,delta chroma_8_std,delta chroma_9_std,delta chroma_10_std,delta chroma_11_std,delta chroma_12_std,delta chroma_std_std
1233,1022_MTI_HAP_XX,1022,HAP,MTI,0.068238,0.036729,2.883470,0.135078,0.165628,0.320623,...,0.008373,0.023914,0.033470,0.014256,0.005344,0.010400,0.023122,0.013638,0.013316,0.011699
3660,1071_TSI_ANG_XX,1071,ANG,TSI,0.308093,0.014928,2.908667,0.364991,0.219840,1.428453,...,0.011577,0.009159,0.009049,0.013339,0.009378,0.009662,0.027554,0.016438,0.009251,0.009637
516,1010_TIE_SAD_XX,1010,SAD,TIE,0.166339,0.005433,2.893361,0.234463,0.202409,1.020516,...,0.008885,0.018680,0.043085,0.025940,0.008065,0.010766,0.012685,0.013108,0.014175,0.010117
816,1016_ITS_ANG_XX,1016,ANG,ITS,0.161589,0.019299,2.959285,0.241155,0.215470,1.062989,...,0.006837,0.016379,0.006640,0.021170,0.011743,0.015311,0.014089,0.019125,0.018590,0.007897
3002,1058_DFA_ANG_XX,1058,ANG,DFA,0.155004,0.011965,2.915512,0.250254,0.215903,1.058873,...,0.030465,0.022781,0.013922,0.011492,0.007407,0.041611,0.015088,0.011237,0.005869,0.015640


In [6]:
# Check the percentages of the different emotion categories in the training set

data_train.Emotion.value_counts(normalize=True)

NEU    0.178718
HAP    0.164359
FEA    0.164359
ANG    0.164359
SAD    0.164103
DIS    0.164103
Name: Emotion, dtype: float64

In [7]:
# Check the percentages of the different emotion categories in the test set

data_test.Emotion.value_counts(normalize=True)

NEU    0.179303
DIS    0.164959
ANG    0.163934
SAD    0.163934
FEA    0.163934
HAP    0.163934
Name: Emotion, dtype: float64

In [8]:
y_train = data_train[['Emotion']]
y_test  = data_test[['Emotion']]

X_train = data_train.drop(columns  = ['FileID','actorID', 'Emotion'])
X_test  = data_test.drop(columns   = ['FileID','actorID', 'Emotion'])

In [9]:
X_train

,SentenceID,zcr_mean,energy_mean,energy_entropy_mean,spectral_centroid_mean,spectral_spread_mean,spectral_entropy_mean,spectral_flux_mean,spectral_rolloff_mean,mfcc_1_mean,...,delta chroma_4_std,delta chroma_5_std,delta chroma_6_std,delta chroma_7_std,delta chroma_8_std,delta chroma_9_std,delta chroma_10_std,delta chroma_11_std,delta chroma_12_std,delta chroma_std_std
837,MTI,0.083640,0.008714,2.907248,0.154361,0.169981,0.577404,0.010233,0.114029,-29.271150,...,0.019546,0.028316,0.014390,0.012027,0.009329,0.017190,0.008330,0.008765,0.013606,0.009506
2490,ITS,0.151667,0.005787,2.860109,0.249394,0.251518,0.661992,0.007877,0.190744,-30.414050,...,0.023129,0.021614,0.032285,0.012211,0.004090,0.012739,0.007861,0.010684,0.006603,0.010285
3885,IOM,0.081441,0.042436,3.050415,0.161888,0.187664,0.368531,0.015429,0.106367,-25.875142,...,0.003848,0.006165,0.037184,0.027610,0.003999,0.026874,0.024719,0.020995,0.012547,0.015755
845,TAI,0.150406,0.010609,2.928441,0.227848,0.218879,0.627350,0.010261,0.211001,-29.907287,...,0.029635,0.032077,0.014713,0.012294,0.004989,0.013047,0.008941,0.010526,0.008577,0.010364
1890,IWL,0.087894,0.021971,3.021540,0.146429,0.162692,0.525216,0.010559,0.115676,-28.654314,...,0.036706,0.020806,0.005194,0.009952,0.025127,0.020504,0.016022,0.008138,0.002314,0.012838
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2046,TIE,0.119205,0.016565,2.871881,0.213218,0.202107,0.942230,0.007878,0.192428,-25.248729,...,0.011450,0.007873,0.011765,0.012555,0.014883,0.008454,0.021950,0.024942,0.023826,0.008226
3271,DFA,0.138713,0.015348,2.759433,0.218334,0.191528,1.045024,0.009202,0.207948,-28.248249,...,0.022454,0.016348,0.030786,0.028595,0.020358,0.004879,0.009590,0.031912,0.014395,0.013876
3148,IOM,0.049920,0.047684,2.882915,0.130239,0.187099,0.211214,0.014590,0.072257,-26.456010,...,0.031786,0.044909,0.045720,0.032407,0.000848,0.009181,0.015284,0.004561,0.001602,0.013600
4337,WSI,0.135008,0.016936,2.900700,0.222885,0.202980,0.792664,0.008293,0.174590,-26.584649,...,0.016742,0.032236,0.036552,0.001970,0.000609,0.001014,0.015637,0.009353,0.003242,0.009500


In [10]:
# Add a column to the y vectors encoding each of the emotions.

y_train_dummies = pd.get_dummies(y_train)
y_train         = pd.concat([y_train, y_train_dummies], axis=1)

y_test_dummies  = pd.get_dummies(y_test)
y_test          = pd.concat([y_test, y_test_dummies], axis=1)


In [26]:
y_train

,Emotion,Emotion_ANG,Emotion_DIS,Emotion_FEA,Emotion_HAP,Emotion_NEU,Emotion_SAD
837,HAP,0,0,0,1,0,0
2490,SAD,0,0,0,0,0,1
3885,SAD,0,0,0,0,0,1
845,SAD,0,0,0,0,0,1
1890,FEA,0,0,1,0,0,0
...,...,...,...,...,...,...,...
2046,ANG,1,0,0,0,0,0
3271,DIS,0,1,0,0,0,0
3148,SAD,0,0,0,0,0,1
4337,HAP,0,0,0,1,0,0


In [11]:
# Fron Sentences

#X_train_temp = X_train.loc[(X_train.SentenceID == 'IOE') | (X_train.SentenceID == 'TIE') | (X_train.SentenceID == 'MTI') | (X_train.SentenceID == 'DFA')]
#X_test_temp  = X_test.loc[(X_test.SentenceID == 'IOE') | (X_test.SentenceID == 'TIE') | (X_test.SentenceID == 'MTI') | (X_train.SentenceID == 'DFA')]

#X_train_sen = X_train_temp.drop(columns  = ['SentenceID'])
#X_test_sen  = X_test_temp.drop(columns   = ['SentenceID'])

#y_train_sen = y_train.loc[(X_train.SentenceID == 'IOE') | (X_train.SentenceID == 'TIE') | (X_train.SentenceID == 'MTI') | (X_train.SentenceID == 'DFA')]
#y_test_sen  = y_test.loc[(X_test.SentenceID == 'IOE') | (X_test.SentenceID == 'TIE') | (X_test.SentenceID == 'MTI') | (X_train.SentenceID == 'DFA')]

# Emb Sentences

#X_train_temp = X_train.loc[(X_train.SentenceID == 'IWW') | (X_train.SentenceID == 'ITH') | (X_train.SentenceID == 'ITS')]
#X_test_temp  = X_test.loc[(X_test.SentenceID == 'IWW') | (X_test.SentenceID == 'ITH') | (X_test.SentenceID == 'ITS')]

#X_train_sen = X_train_temp.drop(columns  = ['SentenceID'])
#X_test_sen  = X_test_temp.drop(columns   = ['SentenceID'])

#y_train_sen = y_train.loc[(X_train.SentenceID == 'IWW') | (X_train.SentenceID == 'ITH') | (X_train.SentenceID == 'ITS')]
#y_test_sen  = y_test.loc[(X_test.SentenceID == 'IWW') | (X_test.SentenceID == 'ITH') | (X_test.SentenceID == 'ITS')]

#  Sentences

X_train_temp = X_train.loc[(X_train.SentenceID == 'TAI') | (X_train.SentenceID == 'IWL') | (X_train.SentenceID == 'IOM') | (X_train.SentenceID == 'TSI') | (X_train.SentenceID == 'WSI')]
X_test_temp  = X_test.loc[(X_test.SentenceID == 'TAI') | (X_test.SentenceID == 'IWL') | (X_test.SentenceID == 'IOM') | (X_train.SentenceID == 'TSI') | (X_train.SentenceID == 'WSI')]

X_train_sen = X_train_temp.drop(columns  = ['SentenceID'])
X_test_sen  = X_test_temp.drop(columns   = ['SentenceID'])

y_train_sen = y_train.loc[(X_train.SentenceID == 'TAI') | (X_train.SentenceID == 'IWL') | (X_train.SentenceID == 'IOM') | (X_train.SentenceID == 'TSI') | (X_train.SentenceID == 'WSI')]
y_test_sen  = y_test.loc[(X_test.SentenceID == 'TAI') | (X_test.SentenceID == 'IWL') | (X_test.SentenceID == 'IOM') | (X_train.SentenceID == 'TSI') | (X_train.SentenceID == 'WSI')]





In [12]:
# Example: try to train a support vector machine with Gaussian radial kernel to distinguish between instances
# where emotion is 'ANG' and instances where emotion is 'HAP'. 

# Get the rows of X_train, X_test corresponding to just the emotions 'ANG' and 'HAP'

X_train_hap = X_train_sen.loc[(y_train_sen.Emotion == 'HAP') | (y_train_sen.Emotion == 'NEU')]
X_test_hap  = X_test_sen.loc[(y_test_sen.Emotion == 'HAP') | (y_test_sen.Emotion == 'NEU')]


# Get the Emotion_ANG column of the ys, with only the rows corresponding to 'ANG' and 'HAP'

y_train_hap = y_train_sen.loc[(y_train_sen.Emotion == 'HAP') | (y_train_sen.Emotion == 'NEU')].Emotion_HAP
y_test_hap  = y_test_sen.loc[(y_test_sen.Emotion == 'HAP') | (y_test_sen.Emotion == 'NEU')].Emotion_HAP


In [13]:
X_train_ang = X_train_sen.loc[(y_train_sen.Emotion == 'ANG') | (y_train_sen.Emotion == 'NEU')]
X_test_ang  = X_test_sen.loc[(y_test_sen.Emotion == 'ANG') | (y_test_sen.Emotion == 'NEU')]

y_train_ang = y_train_sen.loc[(y_train_sen.Emotion == 'ANG') | (y_train_sen.Emotion == 'NEU')].Emotion_ANG
y_test_ang = y_test_sen.loc[(y_test_sen.Emotion == 'ANG') | (y_test_sen.Emotion == 'NEU')].Emotion_ANG

In [14]:
X_train_sad = X_train_sen.loc[(y_train_sen.Emotion == 'SAD') | (y_train_sen.Emotion == 'NEU')]
X_test_sad  = X_test_sen.loc[(y_test_sen.Emotion == 'SAD') | (y_test_sen.Emotion == 'NEU')]

y_train_sad = y_train_sen.loc[(y_train_sen.Emotion == 'SAD') | (y_train_sen.Emotion == 'NEU')].Emotion_SAD
y_test_sad  = y_test_sen.loc[(y_test_sen.Emotion == 'SAD') | (y_test_sen.Emotion == 'NEU')].Emotion_SAD

In [15]:
X_train_dis = X_train_sen.loc[(y_train_sen.Emotion == 'DIS') | (y_train_sen.Emotion == 'NEU')]
X_test_dis  = X_test_sen.loc[(y_test_sen.Emotion == 'DIS') | (y_test_sen.Emotion == 'NEU')]

y_train_dis = y_train_sen.loc[(y_train_sen.Emotion == 'DIS') | (y_train_sen.Emotion == 'NEU')].Emotion_DIS
y_test_dis  = y_test_sen.loc[(y_test_sen.Emotion == 'DIS') | (y_test_sen.Emotion == 'NEU')].Emotion_DIS

In [16]:
X_train_fea = X_train_sen.loc[(y_train_sen.Emotion == 'FEA') | (y_train_sen.Emotion == 'NEU')]
X_test_fea  = X_test_sen.loc[(y_test_sen.Emotion == 'FEA') | (y_test_sen.Emotion == 'NEU')]

y_train_fea = y_train_sen.loc[(y_train_sen.Emotion == 'FEA') | (y_train_sen.Emotion == 'NEU')].Emotion_FEA
y_test_fea  = y_test_sen.loc[(y_test_sen.Emotion == 'FEA') | (y_test_sen.Emotion == 'NEU')].Emotion_FEA

In [17]:
#X_train_sub = X_train
#X_test_sub  = X_test

#y_train_sub = y_train
#y_test_sub  = y_test

In [18]:
print("X_train_sub",  X_train_hap.shape)
print("y_train_sub",  y_train_hap.shape)

X_train_sub (583, 136)
y_train_sub (583,)


In [19]:
np.unique(y_test_hap)

array([0, 1], dtype=uint8)

In [20]:
# Build pipeline to first scale the mid feature data, then apply the SVC

pipe = Pipeline([('scale', StandardScaler()),
                 ('svc', SVC(kernel='rbf'))])


# Fit the model to the training data

pipe.fit(X_train_hap, y_train_hap)
pipe.fit(X_train_ang, y_train_ang)
pipe.fit(X_train_sad, y_train_sad)
pipe.fit(X_train_dis, y_train_dis)
pipe.fit(X_train_fea, y_train_fea)

    
# Get the model's prediction on the test data

pred_hap = pipe.predict(X_test_hap)
pred_ang = pipe.predict(X_test_ang)
pred_sad = pipe.predict(X_test_sad)
pred_dis = pipe.predict(X_test_dis)
pred_fea = pipe.predict(X_test_fea)


In [21]:
# Look at the confusion matrix for the test data :

print("confusion matrix for the happy/neutral test set is:")
print(confusion_matrix(y_test_hap, pred_hap))
print()

# Look at the confusion matrix for the training data:
pred_train_hap = pipe.predict(X_train_hap)
print("confusion matrix for the happy/netural train set is:")
print(confusion_matrix(y_train_hap, pred_train_hap))


confusion matrix for the happy/neutral test set is:
[[27 14]
 [13 30]]

confusion matrix for the happy/netural train set is:
[[279  12]
 [128 164]]


In [22]:
# Look at the confusion matrix for the test data :

print("confusion matrix for the angry/neutral test set is:")
print(confusion_matrix(y_test_ang, pred_ang))
print()

# Look at the confusion matrix for the training data:
pred_train_ang = pipe.predict(X_train_ang)
print("confusion matrix for the angry/netural train set is:")
print(confusion_matrix(y_train_ang, pred_train_ang))


confusion matrix for the angry/neutral test set is:
[[27 14]
 [12 23]]

confusion matrix for the angry/netural train set is:
[[279  12]
 [ 95 205]]


In [23]:
# Look at the confusion matrix for the test data :

print("confusion matrix for the sad/neutral test set is:")
print(confusion_matrix(y_test_sad, pred_sad))
print()

# Look at the confusion matrix for the training data:
pred_train_sad = pipe.predict(X_train_sad)
print("confusion matrix for the sad/netural train set is:")
print(confusion_matrix(y_train_sad, pred_train_sad))


confusion matrix for the sad/neutral test set is:
[[27 14]
 [18 21]]

confusion matrix for the sad/netural train set is:
[[279  12]
 [172 128]]


In [24]:
# Look at the confusion matrix for the test data :

print("confusion matrix for the disgust/neutral test set is:")
print(confusion_matrix(y_test_dis, pred_dis))
print()

# Look at the confusion matrix for the training data:
pred_train_dis = pipe.predict(X_train_dis)
print("confusion matrix for the disgust/netural train set is:")
print(confusion_matrix(y_train_dis, pred_train_dis))

confusion matrix for the disgust/neutral test set is:
[[27 14]
 [30 17]]

confusion matrix for the disgust/netural train set is:
[[279  12]
 [198  86]]


In [25]:
# Look at the confusion matrix for the test data :

print("confusion matrix for the fear/neutral test set is:")
print(confusion_matrix(y_test_fea, pred_fea))
print()

# Look at the confusion matrix for the training data:
pred_train_fea = pipe.predict(X_train_fea)
print("confusion matrix for the fear/netural train set is:")
print(confusion_matrix(y_train_fea, pred_train_fea))


confusion matrix for the fear/neutral test set is:
[[27 14]
 [12 23]]

confusion matrix for the fear/netural train set is:
[[279  12]
 [ 19 276]]
